In [98]:
# 已移除 AIMessageChunk 合并猴补丁，避免递归重入问题。

In [99]:

import logging
logging.basicConfig(level=logging.DEBUG)


In [100]:
import os
import sys
import uuid
from typing import Any, Dict, List, Optional, TypedDict

import importlib.metadata as meta


print(meta.version("langchain"))

# 添加 backend 目录到 Python 路径，以便导入 app 模块
backend_dir = '/root/consult/backend'
if backend_dir not in sys.path:
    sys.path.insert(0, backend_dir)
    print(f"✅ 已将 {backend_dir} 添加到 Python 路径")

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as tool_dec
from app.utils.progress_broker import get_progress_broker
import time
import asyncio

1.0.2


In [101]:
os.environ["HF_ENDPOINT"]="https://hf-mirror.com"
os.environ["HF_HUB_DOWNLOAD_PROGRESS"] = "1"
os.environ['HF_HUB_OFFLINE'] = '1'  # 禁用 HuggingFace Hub 连接

In [102]:
from app.services.llamaindex_retriever import LlamaIndexRetriever


In [103]:
from app.utils.import_with_timeout import import_symbol_with_timeout
LlamaIndexRetriever = import_symbol_with_timeout(
    "app.services.llamaindex_retriever", "LlamaIndexRetriever", timeout_seconds=5.0
)

In [104]:
os.environ["HF_ENDPOINT"]="https://hf-mirror.com"
os.environ["HF_HUB_DOWNLOAD_PROGRESS"] = "1"
os.environ['HF_HUB_OFFLINE'] = '1'  # 禁用 HuggingFace Hub 连接
os.environ['HF_DATASETS_OFFLINE'] = '1'  # 禁用数据集下载
os.environ['TRANSFORMERS_OFFLINE'] = '1'  # 禁用 Transformers 在线功能
os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'  # 禁用遥测（避免联网）

In [105]:
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '1'

In [106]:
os.environ['LOCAL_BGE_MODEL_DIR'] = '/root/consult/backend/models/bge-large-zh-v1.5'

In [107]:
retriever_global = LlamaIndexRetriever.get_instance("global")

In [108]:
from app.services.web_search_service import get_web_search_service
web_search = get_web_search_service()


In [109]:
retriever_workspace = LlamaIndexRetriever.get_instance("global")


In [110]:
from dotenv import load_dotenv
load_dotenv("/root/consult/backend/.env")

True

In [111]:
api_key = os.getenv('THIRD_PARTY_API_KEY') 
api_base = os.getenv('THIRD_PARTY_API_BASE') 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, openai_api_key=api_key, openai_api_base=api_base).with_config({"stream": False})

In [112]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langchain_core.messages import AnyMessage, ToolMessage, HumanMessage, AIMessage, SystemMessage
import logging
import logging, sys
from operator import add
from dataclasses import asdict

from typing import TypedDict, Annotated
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="%(levelname)s:%(name)s:%(message)s",
    force=True,  # 关键
)

class QBState(TypedDict):
    request: Dict[str, Any]
    workspace_id: str
    company_name: Optional[str]
    target_projects: List[str]
    known_info: Dict[str, Any]
    global_db_out: str
    messages: Annotated[list[AnyMessage], add]
    type: str
    retry_count: int
    max_retries: int

from langchain_core.tools import tool

    
class QuestionnaireBuilderWorkflow:
    def __init__(self, workspace_retriever, global_retriever, web_search_service, llm=None):
        self.workspace_retriever = workspace_retriever
        self.global_retriever = global_retriever
        self.web_search_service = web_search_service
        if llm is None:
            api_key = os.getenv('THIRD_PARTY_API_KEY') or os.getenv('OPENAI_API_KEY')
            api_base = os.getenv('THIRD_PARTY_API_BASE') or os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
            self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, openai_api_key=api_key, openai_api_base=api_base).with_config({"stream": False})
        else:
            self.llm = llm.with_config({"stream": False}) if hasattr(llm, "with_config") else llm
        self.graph = self._build_graph()
        self.checkpointer = MemorySaver()
        self.compiled_graph = self.graph.compile(checkpointer=self.checkpointer)

    
    
    

    def _build_graph(self) -> StateGraph:
        
        @tool
        def search_global_db(query: str) -> str:
            """从全局数据库中检索相关数据，全局数据库包含各类政策信息与过去的申请案例，输入具体检索内容"""
            snippets: List[str] = []
            # 全局检索（同步包装）
            try:
                logging.info(f"[db_search_tool] 开始全局检索: query={query}")
                g_results = asyncio.run(self.global_retriever.retrieve(query=query, top_k=6, use_hybrid=True, use_compression=True))
                logging.info(f"[db_search_tool] 全局检索返回 {len(g_results)} 条结果")
                for r in g_results:
                    title = r.get("title") or r.get("name") or r.get("document_id") or "片段"
                    text = r.get("text") or r.get("content") or ""
                    snippets.append(f"[GLOBAL] {title}: {text[:300]}")
            except Exception as e:
                logging.info(f"[db_search_tool] 全局检索失败: {e}")
            return "\n".join(snippets[:12]) or "未找到有效片段"
        
        @tool
        def search_web(query: str) -> str:
            """从网络中检索相关数据；输入中文查询，返回若干条标题与链接摘要"""
            try:
                # 如果服务是异步实现，这里用 asyncio.run 同步封装
                results = asyncio.run(self.web_search_service.search_web(query=query, num_results=6))
            except Exception as e:
                logging.info(f"[web_search_tool] 网络搜索失败: {e}")
                return "(网络搜索失败)"
            if not results:
                return "(未找到结果)"
            lines: List[str] = []
            for idx, r in enumerate(results[:8], 1):
                d = asdict(r)
                title = d.get("title") or r.get("name") or r.get("snippet_title") or "未知标题"
                url = d.get("url") or r.get("link") or r.get("source_url") or ""
                snippet = d.get("snippet") or r.get("content") or r.get("desc") or ""
                lines.append(f"[{idx}] {title}\n{url}\n{snippet[:200]}")
            print("lines","\n\n".join(lines))
            return "\n\n".join(lines)


        def db_search_node(state: QBState) -> QBState:

            logging.info(f"db_search_node")
            tp = ", ".join(state.get("target_projects") or [])
            known = state.get("known_info") 

            react_system_text = (
                "你是中国政策与补贴申报顾问。用户希望申请以下项目：{target_projects}\n"
                "已知信息（可能不完整）：\n{known_info}\n\n"
                "目标：输出面向实操的‘申报条件综述’。\n"
                "执行方式（严格遵守）：\n"
                "1) 如信息不足，循环调用 search_global_db 检索相关片段并综合要点；\n"
                "2) 每轮检索后判断是否仍存在关键空缺（资格/门槛/材料/流程/时间/例外情形）；\n"
                "3) 若仍有空缺，则继续调用 search_global_db；若已覆盖充分则停止循环；\n"
                "4) 最多调用工具 3 次，达到上限后必须仅输出 OK 并停止；\n"
                # "5) 完成后仅输出 OK 作为结束信号（禁止提前总结）。\n\n"
                "注意：优先引用明确口径与阈值，并标注来源类别。"
                
            ).format(
                target_projects=tp or "未指定",
                known_info=known or "无",
            )

            agent = create_agent(
                    model=self.llm,
                    tools=[search_global_db],
                    system_prompt=react_system_text,
                )
            message_content = state.get("messages")

            # 安全地提取消息内容
            def get_content(msg):
                if msg is None:
                    return ""
                if isinstance(msg, dict):
                    return msg.get("content", "")
                if hasattr(msg, "content"):
                    return getattr(msg, "content", "")
                if isinstance(msg, list) and len(msg) > 0:
                    return get_content(msg[0])
                return str(msg) if msg else ""
            logging.info(f"[db_search_tool] 调用开始检索")
            if message_content is not None and len(message_content) > 1:
                res_msg = agent.invoke({"messages": [{"role": "user", "content": "根据这些项目生成条件要求总览,已知总结结果：" + \
                get_content(message_content[-2]) + "，判断结果" + get_content(message_content[-1])}]})
            else:
                res_msg = agent.invoke({"messages": [{"role": "user", "content": "根据这些项目生成条件要求总览"}]})
            prev_msgs = res_msg.get("messages") or []
            logging.info(f"[db_search_tool] 模型检索完成")
            return {"messages": prev_msgs}
        
        def web_search_node(state: QBState) -> QBState:
            """
            使用网络搜索工具对指定项目进行外部信息检索，返回若干检索要点。
            仅返回增量键以满足 LangGraph 的合并规则。
            """
            logging.info("[web_search_node] start")
            tp = ", ".join(state.get("target_projects") or [])
            known = state.get("known_info") or {}
            known_blob = "\n".join([f"- {k}: {v}" for k, v in list(known.items())[:12]]) if isinstance(known, dict) else str(known)

            system_text = (
                "你是网络检索与情报整合助手。目标：围绕‘{target_projects}’项目，为企业用户提供申报所需要点。\n"
                "检索与迭代规则（严格执行）：\n"
                "1) 先生成一批高质量中文检索词（3-8条），覆盖：申报条件/资格门槛/材料清单/办理流程/关键时间/例外情形/资金标准。必要时加入单位/地名限定。\n"
                "2) 强调优先从政府公开网站（如各级政府官网、部门官网、政务公开专栏、政策公告栏、gov.cn 域名等）进行检索，可以先检索官网公告或政策通知的具体发布位置，再深入细分栏目和部门。每轮按检索词调用 search_web 工具获取结果；优先来源顺序：官网/官媒（gov.cn/部门官网/平台）、政策文件与指南、地方主管部门、企查查/企信宝等工商信息、招股书/年报/公告、主流媒体；尽量附原始链接。\n"
                "3) 每轮结束自检信息空缺（上述各类要点是否覆盖、是否有明确阈值/关键口径/例外情形），若仍存在空缺，则生成更精准的新检索词（含实体/年份/地区/文号等），再进行下一轮；最多迭代 8 次。\n"
                "4) 达到 3 次上限后，必须仅输出：OK，并停止检索与输出；\n"
                "5) 输出时先列出‘检索词与命中概览’（按轮次/关键词列举命中情况与链接），再给出‘要点汇总’（分：资格门槛/材料/流程/时间/例外/资金标准），每点后标注来源编号。若信息已充分，请在最后单独一行仅输出：OK。\n"
                "注意：避免泛化与编造，尽量引用来源原文口径；同源重复合并；对工商主体信息可调用与企业相关的公开渠道（如：企查查、企信宝），谨慎标注身份字段。\n\n"
                "已知背景（供参考，可能不完整）：\n{known_info}"
            ).format(target_projects=tp or "未指定", known_info=known_blob or "无")

            agent = create_agent(
                model=self.llm,
                tools=[search_web],
                system_prompt=system_text,
            )

            # 生成一次性检索提示（也可根据需要改为多轮）
            message_content = state.get("messages")
            def get_content(msg):
                if msg is None:
                    return ""
                if isinstance(msg, dict):
                    return msg.get("content", "")
                if hasattr(msg, "content"):
                    return getattr(msg, "content", "")
                if isinstance(msg, list) and len(msg) > 0:
                    return get_content(msg[0])
                return str(msg) if msg else ""
            logging.info(f"[db_search_tool] 调用开始检索")
            if message_content is not None and len(message_content) > 1:
                res_msg = agent.invoke({"messages": [{"role": "user", "content": "请检索并汇总要点，给出链接与简要摘录,已知总结结果：" + \
                get_content(message_content[-2]) + "，判断结果" + get_content(message_content[-1])}]})
            else:
                res_msg = agent.invoke({"messages": [{"role": "user", "content": "请检索并汇总要点，给出链接与简要摘录"}]})
            prev_msgs = res_msg.get("messages") or []

            print(prev_msgs)    

            return {"messages": [prev_msgs]}


        def summery_node(state: QBState) -> QBState:

            logging.info(f"[summery_node] 开始总结")

            prompt = (
                "你是政策要求材料总结助手。用户希望申请以下项目：{target_projects}\n"
                "已知信息（可能不完整）：\n{known_info}\n\n"
                "你已经从数据库或网络中检索到了相关信息，请你基于这些信息，系统性、详细地总结申报条件要求和必要材料，涵盖但不限于以下方面：资格要求、申报门槛、必需材料、办理流程、关键时间节点、常见例外等。\n"
                "请充分整合所有获取到的政策条款、申报文件、案例等数据，对每项条件尽量详细具体，必要时举例说明，并按类别或逻辑结构清晰归纳，帮助用户快速把握核心要点和潜在难点。\n"
                "在总结后，请进一步思考和指出：\n"
                "1）根据当前信息，还有哪些可能的申报条件、材料或案例尚未被覆盖，建议检索哪些内容以补全信息？\n"
                "2）请列出你的补全建议点和你的分析思路，帮助后续继续完善。"
                "尽量细化到每个条件的具体内容要求"
            )

            # 直接从 QBState 获取 ToolMessage

            msgs = state.get("messages") or []
            tool_texts = [m.content for m in msgs if isinstance(m, ToolMessage)]
            tool_blob = "\n\n".join(tool_texts) if tool_texts else "（无工具输出）"
            # print("tool_blob",tool_blob)
            prompts = [
                SystemMessage(content=prompt),
                HumanMessage(content=tool_blob),
            ]
            res_msg = self.llm.invoke(prompts)
            logging.info(f"[summery_node] 总结完成")
            # print("res_msg",res_msg.content)
            print("[summery node] out", res_msg)
            return {"messages": [res_msg]}
        
        def Judger_node(state: QBState) -> QBState:
            """
            根据总结信息，判断是否需要进行检索，如果要检索具体哪些方案还要检索；受 max_retries 控制。
            """
            logging.info(f"[Judger_node] 开始判断")
            # 已达最大次数则直接放弃检索
            retry_count = int(state.get("retry_count") or 0)
            max_retries = int(state.get("max_retries") or 0)
            logging.info(f"retry_count: {retry_count}, max_retries: {max_retries}")
            if retry_count >= max_retries and max_retries > 0:
                logging.info(f"[Judger_node] 已达最大重检索次数: {retry_count}/{max_retries}")
                return {"type": "放弃检索"}

            prompt = (
                "你是申报条件判定助手。用户希望申请以下项目：{target_projects}\n"
                "已知信息（可能不完整）：\n{known_info}\n\n"
                "你已经获得了对申报条件与材料的总结（见下文），请根据这些内容判断：\n"
                "你的回答前四个字必须为“库中检索”或“网络检索”或“放弃检索”，用于直接表示是否还需要进一步检索信息；\n"
                "之后再给出具体建议方案和理由，但务必保证前四个字为“库中检索”或“网络检索”或“放弃检索”并直接作答；\n"
                "例如：\n"
                "库中检索，需要进一步检索学历要求、财务指标明细，因为…\n"
                "放弃检索，所有关键信息均已覆盖，无需补充。\n"
                "网络检索，需要进一步检索学历要求、财务指标明细，因为…\n"
                "请严格遵循：前四个字只能为“库中检索”或“网络检索”或“放弃检索”。"
                "若需要检索, 请列出需检索的具体方案（如：学历要求、财务指标明细等）及你的判定理由。\n"
            ).format(
                target_projects=state.get("target_projects") or "未指定",
                known_info=state.get("known_info") or "无"
            )

            logging.info("总结结果" + state.get("messages")[-1].content)
            # print(state.get("messages"))

            prompts = [
                SystemMessage(content=prompt),
                HumanMessage(content=state.get("messages")[-1].content),
            ]
            res_msg = self.llm.invoke(prompts)
            logging.info("[Judger_node] 模型判断完成")
            # 取前3字作为动作标记
            type_now = res_msg.content[0:4]
            print("type_now",type_now)
            print("res_msg",res_msg.content)
            logging.info(f"[Judger_node] 判断结果 {type_now}")
            # 若需要再次检索，则递增 retry_count
            print("[Judger node] out", res_msg)
            if type_now in ("库中检索", "网络检索"):
                return {"messages": [res_msg], "type": type_now, "retry_count": retry_count + 1}
            else:
                return {"messages": [res_msg], "type": type_now}
        
        def person_info_web_search_node(state: QBState) -> QBState:
            """
            使用网络搜索工具对指定项目进行外部信息检索，返回若干检索要点。
            仅返回增量键以满足 LangGraph 的合并规则。
            """
            logging.info("[web_search_node] start")
            company_name = ", ".join(state.get("company_name") or [])
            known = state.get("known_info") or {}
            known_blob = "\n".join([f"- {k}: {v}" for k, v in list(known.items())[:12]]) if isinstance(known, dict) else str(known)

            system_text = (
                "你是“主体信息核验”检索助手。任务：围绕以下“公司/个人”实体，检索公开渠道并给出可佐证材料与链接，验证其是否满足相关政策申报所需的主体条件与记录。\n"
                "检索与迭代规则（严格执行）：\n"
                "1) 先生成一批中文检索词（3-8条），覆盖：企业基础信息（名称/统一社会信用代码/注册地/高管股东）、资格资质/行政许可、行政处罚/信用记录、司法文书/裁判/执行、知识产权、公告/年报/招股书/招投标、主流媒体报道等；必要时加入实体/地区/年份/文号限定。\n"
                "2) 每轮按检索词调用 search_web 获取结果；优先来源顺序：政府/主管部门官网与平台（gov.cn、地方政务、信用平台、司法/裁判文书网）> 政策文件/公告/年报/招股书/招投标/公示 > 工商主体公开库（企查查、企信宝等）> 主流媒体/权威行业平台；尽量提供原始链接。\n"
                "3) 每轮完成后自检覆盖度：上述要点是否覆盖？是否有明确口径/阈值/例外？若仍有空缺，则生成更精准的新检索词（含实体/地区/年份/文号/关键词）并进行下一轮；最多迭代 8 次。\n"
                "4) 达到 8 次上限后，必须仅输出：OK，并停止检索与输出。\n"
                "输出格式（严格遵循）：\n"
                "- 检索词与命中概览（按轮次/关键词列出命中与链接，1-2 行要点）\n"
                "- 证据要点汇总（分项：基础信息、资格/许可、处罚/信用、司法/裁判、知识产权、公告/年报/招股/招投标、媒体报道等），每点后标注来源编号\n"
                "- 若信息已充分，请在最后单独一行仅输出：OK\n"
                "约束：严禁编造；尽量引用来源原文口径；同源重复合并；对个人敏感信息需最小化披露；如无公开记录，明确说明“未检出”；中文输出。\n\n"
                "已知背景（供参考，可能不完整）：\n{known_info}\n"
                "主体名称：\n{company_name}（如为公司/个人，请在检索词中加入其名称/别名/代码/地区等）"
            ).format(company_name=company_name or "未指定", known_info=known_blob or "无")

            agent = create_agent(
                model=self.llm,
                tools=[search_web],
                system_prompt=system_text,
            )

            # 生成一次性检索提示（也可根据需要改为多轮）
            message_content = state.get("messages")
            def get_content(msg):
                if msg is None:
                    return ""
                if isinstance(msg, dict):
                    return msg.get("content", "")
                if hasattr(msg, "content"):
                    return getattr(msg, "content", "")
                if isinstance(msg, list) and len(msg) > 0:
                    return get_content(msg[0])
                return str(msg) if msg else ""
            logging.info(f"[web_search_tool] 调用开始检索")

            retry_count = int(state.get("retry_count") or 0)
            max_retries = int(state.get("max_retries") or 0)
            logging.info(f"retry_count: {retry_count}, max_retries: {max_retries}")
            message_list = []
            if retry_count >= max_retries and max_retries > 0:
                message_list = [message_content[-1],message_content[-3]]
                res_msg = agent.invoke({"messages": [{"role": "user", "content": "请检索并汇总要点，给出链接与简要摘录,已知总结结果：" + \
                get_content(message_content[-1]) + "，数据库和网络检索已达最大字数，最后一次检索建议：" + get_content(message_content[-3])}]})
            elif message_content is not None and len(message_content) > 1:
                message_list = [message_content[-2],message_content[-1]]
                res_msg = agent.invoke({"messages": [{"role": "user", "content": "请检索并汇总要点，给出链接与简要摘录,已知总结结果：" + \
                get_content(message_content[-2]) + "，判断结果" + get_content(message_content[-1])}]})
            else:
                res_msg = agent.invoke({"messages": [{"role": "user", "content": "请检索并汇总要点，给出链接与简要摘录"}]})
            prev_msgs = res_msg.get("messages") or []

            print("==============person===============")
            print("prev_msgs",prev_msgs)    
            print("message_content[-1]",message_content[-1])
            print("message_content[-3]",message_content[-3])
            
            return {"messages": message_list + [prev_msgs]}
        
        def analysis_node(state: QBState) -> QBState:

            logging.info(f"[analysis_node] 开始分析")

            prompt = (
                "你是“申报可行性评估”专家，面向具体申请人（公司/个人）与目标政策。\n"
                "目标：基于已检索/整理的信息，给出申请成功可能性的系统评估与改进建议，并产出一份进一步核实问卷。\n\n"

                "工作流程（严格遵循）：\n"
                "1) 要求与材料大纲：\n"
                "   - 资格条件（主体/行业/规模/资质/信用/地域/时间窗口）\n"
                "   - 硬性门槛（营收/投资/纳税/研发/人员/社保/不良记录等阈值）\n"
                "   - 佐证材料（营业执照、财务报表、纳税/社保证明、合同、专利/商标、获奖/荣誉等）\n"
                "   - 流程与关键时间点（申报节点、审核、异议、公示、发放）\n"
                "   - 例外情形与排除条款\n"
                "   为每条在括号中标注来源编号（若有）。\n\n"

                "2) 匹配与证据：将申请人信息与要求逐条比对，分组输出：\n"
                "   - 已满足（列证据点与来源编号）\n"
                "   - 基本满足但需补充（缺失/证据弱项）\n"
                "   - 未满足（关键差距）\n"
                "   - 不确定/模糊（待核实）\n\n"

                "3) 评分与结论：\n"
                "   - 从适配度(40)、合规与信用(20)、材料完备度(20)、流程与时间把控(10)、风险与不确定性(10) 五维度打分并给总分（0-100）。\n"
                "   - 给出一句话结论（可行/存在较大不确定/暂不具备），以及三条内的优先行动建议。\n\n"

                "4) 进一步核实问卷：用于向申请人收集关键信息与材料链接（可复制到表单）。\n"
                "   - 每个问题包含：提问、所需证据/证明、说明（为什么需要/判定依据）、期望格式（文件/截图/链接/编号）。\n"
                "   - 优先覆盖“不确定/模糊”和“基本满足但需补充”的条目，按优先级排序。\n"
                "   - 至少给出 8-15 个问题，禁用长篇开放题，尽量结构化。\n\n"

                "输出格式（严格遵循，中文）：\n"
                "## 一、要求与材料大纲（附来源编号）\n"
                "- ...\n\n"
                "## 二、匹配与证据\n"
                "### 2.1 已满足\n"
                "- 要求A：证据...（来源#1）\n"
                "### 2.2 基本满足但需补充\n"
                "- 要求B：缺失...（建议补充...）\n"
                "### 2.3 未满足\n"
                "- 要求C：差距...\n"
                "### 2.4 不确定/模糊\n"
                "- 要求D：原因...（需核实...）\n\n"
                "## 三、评分与结论（总分：X/100）\n"
                "- 适配度：x/40；合规与信用：x/20；材料完备度：x/20；流程与时间：x/10；风险与不确定性：x/10\n"
                "- 结论：...\n"
                "- 优先行动建议：1) ... 2) ... 3) ...\n\n"
                "## 四、进一步核实问卷\n"
                "1) [高优先级] 近两年纳税证明与社保缴纳清单\n"
                "   - 证据：税务/社保官方出具文件或截图\n"
                "   - 说明：核验硬性门槛与在地贡献\n"
                "   - 格式：PDF/官方链接\n"
                "2) ...（按此模板列 8-15 条）\n\n"

                "约束：不得编造；如无来源请标注“无明确来源”；仅提炼必要细节；可引用先前‘检索命中’与‘总结’的来源编号。"
            )

            # 直接从 QBState 获取 ToolMessage

            msgs = state.get("messages") or []
            print("msgs[-1]",msgs[-1])
            print("msgs[-2]",msgs[-2])
            print("msgs[-3]",msgs[-3])
            texts = [msg.content for msg in msgs[-1]]+[msgs[-2].content, msgs[-3].content]
            print("texts",len(texts))
            blob = "\n\n".join(texts) if texts else "（无工具输出）"
            print("blob",blob)


            agent = create_agent(
                model=self.llm,
                tools=[],  # 或不传，视你的封装实现而定
                system_prompt=prompt,
            )
            res_msg = agent.invoke(HumanMessage(content=blob))
            logging.info(f"[analysis_node] 分析完成")
            res_list = res_msg.get("messages")
            # for i, msg in enumerate(res_list):
            #     print(f"res_list[{i}]:", msg)
            
            return {"messages": [res_list[0]]}
        
        



        def router_func(state: QBState):
            if state["type"] == "库中检索":
                return "db_search"
            elif state["type"] == "网络检索":
                return "web_search"
            else:
                return "person_info_web_search"
            
        graph = StateGraph(QBState)      
        graph.add_node("db_search", db_search_node)
        graph.add_node("web_search", web_search_node)
        graph.add_node("summery", summery_node)
        graph.add_node("judger", Judger_node)
        graph.add_node("person_info_web_search", person_info_web_search_node)
        graph.add_node("analysis", analysis_node)



        graph.add_edge(START, "db_search")
        graph.add_edge("db_search", "summery")
        graph.add_edge("summery", "judger")
        graph.add_edge("web_search", "summery")
        graph.add_edge("summery", "judger")
        graph.add_conditional_edges("judger", router_func, ["db_search","web_search","person_info_web_search"])
        graph.add_edge("person_info_web_search", "analysis")
        graph.add_edge("analysis", END)
    
        return graph

    async def run(self, request: Dict[str, Any], phase: Optional[str] = None) -> Dict[str, Any]:
        initial: QBState = {
            "request": request,
            "workspace_id": (request.get("workspace_id") or "global"),
            "company_name": request.get("company_name"),
            "target_projects": request.get("target_projects", []),
            "known_info": request.get("known_info", {}),
            "global_db_out": "",
            "type": "",
            "messages": [],
            "retry_count": int(request.get("retry_count", 0)),
            "max_retries": int(request.get("max_retries", 2)),
        }
        
        # 使用 ainvoke 异步调用，因为我们使用了异步节点
        result: QBState = await self.compiled_graph.ainvoke(
            initial,
            config={"configurable": {"thread_id": str(uuid.uuid4())}, "recursion_limit": 100}
        )

        # 相位裁剪已移除，直接返回
        return result
    

In [113]:
workflow = QuestionnaireBuilderWorkflow(
                    workspace_retriever=retriever_workspace,
                    global_retriever=retriever_global,
                    web_search_service=web_search,
                    llm=None
                )
request_context = {
                    "workspace_id": "global",
                    "company_name": "紫荆思源有限公司",
                    "target_projects": ["前海十二条"],
                    "known_info": {},
                }
result = await workflow.run(request_context)
print(result)

INFO:root:db_search_node
INFO:root:[db_search_tool] 调用开始检索


INFO:httpx:HTTP Request: POST https://api.qingyuntop.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[db_search_tool] 开始全局检索: query=前海十二条 申报条件


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[db_search_tool] 全局检索返回 6 条结果
INFO:httpx:HTTP Request: POST https://api.qingyuntop.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[db_search_tool] 开始全局检索: query=前海十二条 申报条件 资格 材料 流程


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[db_search_tool] 全局检索返回 6 条结果
INFO:httpx:HTTP Request: POST https://api.qingyuntop.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[db_search_tool] 开始全局检索: query=前海十二条 申报条件 例外情形


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[db_search_tool] 全局检索返回 6 条结果
INFO:httpx:HTTP Request: POST https://api.qingyuntop.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[db_search_tool] 模型检索完成
INFO:root:[summery_node] 开始总结
INFO:httpx:HTTP Request: POST https://api.qingyuntop.top/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[summery_node] 总结完成
[summery node] out content='### 申报条件要求和必要材料总结\n\n#### 一、资格要求\n1. **申请单位资格**：\n   - 申请单位必须在前海合作区内设立，且具备合法的营业执照。\n   - 申请单位需在前海合作区内有固定的生产经营场所，面积需填写具体数值。\n   - 申请单位需在前海合作区内有满足生产经营需要的从业人员，需填写具体人数。\n\n2. **申报人员资格**：\n   - 申报人员需为单位法定代表人或授权代理人，需提供身份证明文件。\n\n#### 二、申报门槛\n- 申请单位需在前海合作区内实际经营，且符合《十二条措施》及相关申报指南的要求。\n- 申报单位需在申报年度内按时足额缴纳相关社会保险。\n\n#### 三、必需材料\n1. **基本材料**：\n   - 营业执照复印件。\n   - 申请单位的实际经营承诺书。\n   - 主要生产经营地点的租赁合同或产权证明。\n   - 从业人员的社保缴纳证明。\n\n2. **补充材料**：\n   - 相关的财务报表或审计报告。\n   - 其他与申请相关的证明材料，如行业资质证书等。\n\n#### 四、办理流程\n1. **登录平台**：\n   - 登录前海企业服务一体化服务平台（网址：https://qhsk.sz.gov.cn）。\n   - 选择“法人登录”或“账号密码登录”，首次使用需先注册。\n\n2. **填写申请**：\n   - 根据系统提示逐项填写企业相关信息，确保信息真实、准确。\n   - 